# ML localization 5G NSA: XGBoost and Random Forest

## Data preparation

In [67]:
import pandas as pd
import numpy as np


In [68]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [69]:
nr_df = pd.read_csv('/content/drive/MyDrive/localization/5G_fingerprint_DB_grid.csv', sep=',')
nr_df

,Timestamp,Lat,Lon,BMAVAL1,BMAXML1,BMAXML2,BMAXML3,BMAXMQ1,BMAXMQ2,BMAXMQ3,...,TBTS5L3,TBTS5Q1,TBTS5Q2,TBTS5Q3,TBTS6K1,TBTS6K2,TBTS6L1,TBTS6L2,TBTS6Q1,TBTS6Q2
0,1701853010,49.227139,16.575808,0.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
1,1701853011,49.227137,16.575807,0.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
2,1701853012,49.227134,16.575806,0.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
3,1701853013,49.227133,16.575806,0.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
4,1701853014,49.227133,16.575805,0.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29267,1701951192,49.230384,16.571568,0.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
29268,1701951194,49.230385,16.571568,0.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
29269,1701951196,49.230386,16.571568,0.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
29270,1701951198,49.230387,16.571568,0.0,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0


In [70]:
nr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29272 entries, 0 to 29271
Columns: 1391 entries, Timestamp to TBTS6Q2
dtypes: float64(592), int64(799)
memory usage: 310.6 MB


In [71]:
nr_df.isnull().sum()

Timestamp    0
Lat          0
Lon          0
BMAVAL1      0
BMAXML1      0
            ..
TBTS6K2      0
TBTS6L1      0
TBTS6L2      0
TBTS6Q1      0
TBTS6Q2      0
Length: 1391, dtype: int64

In [72]:
X = nr_df.iloc[:, 3:]
y = nr_df.iloc[:, 1:3]

In [73]:
X

,BMAVAL1,BMAXML1,BMAXML2,BMAXML3,BMAXMQ1,BMAXMQ2,BMAXMQ3,BMBACK3,BMBACL1,BMBACL2,...,TBTS5L3,TBTS5Q1,TBTS5Q2,TBTS5Q3,TBTS6K1,TBTS6K2,TBTS6L1,TBTS6L2,TBTS6Q1,TBTS6Q2
0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,...,0,0,0,0,0.0,0.0,0,0,0,0
1,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,...,0,0,0,0,0.0,0.0,0,0,0,0
2,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,...,0,0,0,0,0.0,0.0,0,0,0,0
3,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,...,0,0,0,0,0.0,0.0,0,0,0,0
4,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,...,0,0,0,0,0.0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29267,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,...,0,0,0,0,0.0,0.0,0,0,0,0
29268,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,...,0,0,0,0,0.0,0.0,0,0,0,0
29269,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,...,0,0,0,0,0.0,0.0,0,0,0,0
29270,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,...,0,0,0,0,0.0,0.0,0,0,0,0


In [74]:
import sklearn
from sklearn.model_selection import train_test_split

In [75]:
# Train/test split 8/2
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, shuffle=True, test_size=0.2)

# Confirm how the data was split
# print(X_train.shape)
# print(X_test.shape)
# print(y_train.shape)
# print(y_test.shape)

## XGBoost

In [76]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor()

In [77]:
# Train the model using the training sets
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [78]:
xgb_model.get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': None,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [79]:
#run the predictions on the testing data
y_xgb_pred_test = xgb_model.predict(X_test)

In [80]:
# Determine accuracy uisng 𝑅^2
from sklearn.metrics import r2_score

In [82]:
# R^2 score
score_xgb = r2_score(y_test, y_xgb_pred_test)

print("R^2 - {}%".format(round(score_xgb, 5) *100))

R^2 - 96.90299999999999%


In [83]:
# MSE and RMSE
from sklearn.metrics import mean_squared_error

In [93]:
mse_xgb = mean_squared_error(y_test, y_xgb_pred_test)
rmse_xgb = mean_squared_error(y_test, y_xgb_pred_test, squared = False)

print(mse_xgb)
print(rmse_xgb)

2.3383921050103316e-05
0.004567538584885896


In [94]:
y_test

,Lat,Lon
644,49.227242,16.587769
19538,49.198667,16.604542
23107,49.191356,16.595758
9962,49.182467,16.690535
19788,49.197731,16.608744
...,...,...
1879,49.223376,16.575539
15204,49.191808,16.616882
20912,49.198329,16.614069
24180,49.199760,16.567983


In [95]:
y_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5855 entries, 644 to 20782
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Lat     5855 non-null   float64
 1   Lon     5855 non-null   float64
dtypes: float64(2)
memory usage: 137.2 KB


In [96]:
y_xgb_pred_test

array([[49.223988, 16.5914  ],
       [49.197807, 16.606215],
       [49.191505, 16.595493],
       ...,
       [49.197666, 16.613426],
       [49.198566, 16.570194],
       [49.19979 , 16.620176]], dtype=float32)

In [97]:
len(y_xgb_pred_test)

5855

In [98]:
y_test_arr = y_test.to_numpy()

In [99]:
y_test_arr

array([[49.227242, 16.587769],
       [49.198667, 16.604542],
       [49.191356, 16.595758],
       ...,
       [49.198329, 16.614069],
       [49.19976 , 16.567983],
       [49.19987 , 16.621127]])

In [101]:
from math import acos, sin, cos, radians

geo_dist_xgb = np.empty(len(y_xgb_pred_test), dtype=float)
for i in range(0, len(y_xgb_pred_test)):
  geo_dist_xgb[i] = acos(sin(radians(y_test_arr[i, 0]))*sin(radians(y_xgb_pred_test[i, 0]))+cos(radians(y_test_arr[i, 0]))*cos(radians(y_xgb_pred_test[i, 0]))*cos(radians(y_xgb_pred_test[i, 1]) - radians(y_test_arr[i, 1]))) * 6371


In [104]:
geo_dist_xgb

array([0.44775763, 0.1546167 , 0.02541767, ..., 0.08725592, 0.20839252,
       0.06963121])

In [111]:
geo_dist_xgb_m = [i * 1000 for i in geo_dist_xgb]


In [121]:
# Get average of a list
def Average(lst):
    return sum(lst) / len(lst)

average_error_xgb = Average(geo_dist_xgb_m)

# Printing average of the list
print("Average error XGB [m] =", round(average_error_xgb, 5))

Average error XGB [m] = 401.03993


In [110]:
print("Max error XGB [m] = ", np.max(geo_dist_xgb_m))
print("Min error XGB [m] = ", np.min(geo_dist_xgb_m))

Max error XGB [m] =  4131.679123918753
Min error XGB [m] =  2.3904097236944066


## Random forest


In [112]:
from sklearn.ensemble import RandomForestRegressor

In [113]:
rf_model = RandomForestRegressor(n_estimators=10,random_state=10)

# Train the model using the training sets
rf_model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, random_state=10)

In [114]:
#run the predictions on the training and testing data
y_rf_predict_test = rf_model.predict(X_test)

In [115]:
score_rf = r2_score(y_test, y_rf_predict_test)

print("R^2 - {}%".format(round(score_rf, 5) *100))

R^2 - 99.74499999999999%


In [116]:
# RMSE and MSE

mse_rf = mean_squared_error(y_test, y_rf_predict_test)
rmse_rf = mean_squared_error(y_test, y_rf_predict_test, squared = False)

print(mse_rf)
print(rmse_rf)

1.89056986544295e-06
0.001303204616936839


In [118]:
geo_dist_rf = np.empty(len(y_xgb_pred_test), dtype=float)
for i in range(0, len(y_xgb_pred_test)):
  geo_dist_rf[i] = acos(sin(radians(y_test_arr[i, 0]))*sin(radians(y_rf_predict_test[i, 0]))+cos(radians(y_test_arr[i, 0]))*cos(radians(y_rf_predict_test[i, 0]))*cos(radians(y_rf_predict_test[i, 1]) - radians(y_test_arr[i, 1]))) * 6371

In [119]:
geo_dist_rf_m = [i * 1000 for i in geo_dist_rf]
geo_dist_rf_m

[23.40766413740088,
 13.545507242783794,
 41.61784155993869,
 13.958071835371221,
 2.3979385918601723,
 3.8562900331569088,
 15.183119469983568,
 29.208615241388053,
 5.476705354686208,
 33.590866743734445,
 494.7528257466547,
 6.96982863572934,
 0.3002117719193212,
 33.27695486582682,
 2.097189278617512,
 3.470011762034602,
 47.32350822171916,
 29.90627641382171,
 10.689815724879498,
 1.4645904445867268,
 0.484076936857397,
 1.5771836230352876,
 1.3358580532298292,
 8.355384961567468,
 12.258790310527235,
 232.9458998974266,
 23.70494659248213,
 1.8554933375141707,
 154.3707497324193,
 486.1551981350841,
 20.460924450430607,
 6.4604847894188255,
 145.41872116445748,
 5.464349081093755,
 9.647500423971051,
 213.2344008002692,
 4.155520893052667,
 27.88545610877619,
 45.30294116212408,
 53.95123786569047,
 2.618906817494528,
 10.585179292969306,
 29.888490578574174,
 142.65487226262192,
 127.31265914799391,
 120.4020635841227,
 23.590991753084083,
 128.02953306050668,
 20.93662347333093

In [120]:
average_error_rf = Average(geo_dist_rf_m)

# Printing average of the list
print("Average RF error [m] =", round(average_error_rf, 5))
print("Max RF error [m] =", np.max(geo_dist_rf_m))
print("Min RF error [m] =", np.min(geo_dist_rf_m))

Average RF error [m] = 57.08174
Max RF error [m] = 3122.097524763461
Min RF error [m] = 0.0


# Cross check

In [126]:
cross_check_df = pd.read_csv('/content/drive/MyDrive/localization/5G_fingerprint_DB_circle.csv', sep=',')
cross_check_df

,Timestamp,Lat,Lon,BMAVAL1,BMAXML1,BMAXML2,BMAXML3,BMAXMQ1,BMAXMQ2,BMAXMQ3,...,TBTS5L3,TBTS5Q1,TBTS5Q2,TBTS5Q3,TBTS6K1,TBTS6K2,TBTS6L1,TBTS6L2,TBTS6Q1,TBTS6Q2
0,1701439521,49.227657,16.575590,0.0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
1,1701439522,49.227656,16.575592,0.0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
2,1701439523,49.227656,16.575593,0.0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
3,1701439524,49.227654,16.575595,0.0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
4,1701439525,49.227653,16.575596,0.0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7368,1701447326,49.197588,16.608551,0.0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
7369,1701447327,49.197582,16.608546,0.0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
7370,1701447328,49.197578,16.608546,0.0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0
7371,1701447329,49.197579,16.608549,0.0,0,0,0,0,0,0,...,0,0,0,0,0.0,0.0,0,0,0,0


In [127]:
X_check = cross_check_df.iloc[:, 3:]
y_check = cross_check_df.iloc[:, 1:3]

## XGboost cross check

In [128]:
# Run the predictions on the cross-check data
y_xgb_cross = xgb_model.predict(X_check)

In [130]:
# R^2 score
score_xgb_cross = r2_score(y_check, y_xgb_cross)

print("R^2 - {}%".format(round(score_xgb_cross, 5) *100))

R^2 - 90.716%


In [131]:
mse_xgb_cross = mean_squared_error(y_check, y_xgb_cross)
rmse_xgb_cross = mean_squared_error(y_check, y_xgb_cross, squared = False)

print(mse_xgb_cross)
print(rmse_xgb_cross)

3.790602594983051e-05
0.006078826195165555


In [132]:
y_check

,Lat,Lon
0,49.227657,16.575590
1,49.227656,16.575592
2,49.227656,16.575593
3,49.227654,16.575595
4,49.227653,16.575596
...,...,...
7368,49.197588,16.608551
7369,49.197582,16.608546
7370,49.197578,16.608546
7371,49.197579,16.608549


In [134]:
y_check.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7373 entries, 0 to 7372
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Lat     7373 non-null   float64
 1   Lon     7373 non-null   float64
dtypes: float64(2)
memory usage: 115.3 KB


In [135]:
y_xgb_cross

array([[49.225933, 16.578272],
       [49.225933, 16.578272],
       [49.225933, 16.578272],
       ...,
       [49.19851 , 16.607883],
       [49.198074, 16.607422],
       [49.198074, 16.606705]], dtype=float32)

In [137]:
y_check_arr = y_check.to_numpy()

In [138]:
y_check_arr

array([[49.227657, 16.57559 ],
       [49.227656, 16.575592],
       [49.227656, 16.575593],
       ...,
       [49.197578, 16.608546],
       [49.197579, 16.608549],
       [49.197576, 16.608545]])

In [139]:
# Defining the function to calculate geographical distance from coordinates
def geo_distance(true_coord, pred_coord):
  geo_dist = np.empty(len(true_coord), dtype=float)
  for i in range(0, len(true_coord)):
    geo_dist[i] = acos(sin(radians(true_coord[i, 0]))*sin(radians(pred_coord[i, 0]))+cos(radians(true_coord[i, 0]))*cos(radians(pred_coord[i, 0]))*cos(radians(pred_coord[i, 1]) - radians(true_coord[i, 1]))) * 6371
  return geo_dist

# geo_dist_xgb_check = np.empty(len(y_check), dtype=float)
# for i in range(0, len(y_check)):
#   geo_dist_xgb_check[i] = acos(sin(radians(y_check_arr[i, 0]))*sin(radians(y_xgb_pred_test[i, 0]))+cos(radians(y_test_arr[i, 0]))*cos(radians(y_xgb_pred_test[i, 0]))*cos(radians(y_xgb_pred_test[i, 1]) - radians(y_test_arr[i, 1]))) * 6371


In [140]:
geo_dist_xgb_check = geo_distance(y_check_arr, y_xgb_cross)
geo_dist_xgb_check

array([0.27326475, 0.27308326, 0.27303151, ..., 0.11419023, 0.09869592,
       0.1447429 ])

In [142]:
geo_dist_xgb_check_m = [i * 1000 for i in geo_dist_xgb_check]


In [143]:
average_error_xgb_check = Average(geo_dist_xgb_check_m)

# Printing average of the list
print("Average error XGB cross check [m] =", round(average_error_xgb_check, 5))

Average error XGB cross check [m] = 552.11701


In [144]:
print("Max error XGB cross check [m] = ", np.max(geo_dist_xgb_check_m))
print("Min error XGB cross check [m] = ", np.min(geo_dist_xgb_check_m))

Max error XGB cross check [m] =  3468.489058937358
Min error XGB cross check [m] =  7.173742305617015


## RF cross check

In [145]:
#run the predictions on the training and testing data
y_rf_cross = rf_model.predict(X_check)

In [146]:
score_rf_cross = r2_score(y_check, y_rf_cross)

print("R^2 - {}%".format(round(score_rf_cross, 5) *100))

R^2 - 95.005%


In [148]:
# RMSE and MSE

mse_rf_cross = mean_squared_error(y_check, y_rf_cross)
rmse_rf_cross = mean_squared_error(y_check, y_rf_cross, squared = False)

print(mse_rf_cross)
print(rmse_rf_cross)

2.045716383857551e-05
0.004454816314174407


In [151]:
# Distance error
geo_dist_rf_check = geo_distance(y_check_arr, y_rf_cross)

In [152]:
geo_dist_rf_check_m = [i * 1000 for i in geo_dist_rf_check]
geo_dist_rf_check_m

[42.06844325732992,
 53.08912243915051,
 37.72146279864818,
 25.0373738035087,
 26.310269206103367,
 33.58375581918926,
 33.253383488948714,
 24.86616249320153,
 24.635298249544462,
 24.535035501022904,
 24.354558570488123,
 59.03762007494791,
 46.96304395995229,
 24.15576375541512,
 24.106276241789175,
 50.7272620129895,
 41.590979596900226,
 37.114723744892295,
 24.096927578126085,
 24.13318015928769,
 24.085891487928333,
 25.18846540522376,
 25.218861003347076,
 25.30003426321993,
 24.57742670076009,
 25.494856151357517,
 25.626024799284636,
 24.588242173210148,
 40.15504997346097,
 24.93420976087013,
 25.108187808597243,
 33.82975085327721,
 16.369113967499636,
 25.934891690481514,
 25.63534319422985,
 26.850750226535123,
 55.184444163339855,
 52.55824218950566,
 52.58198686757977,
 31.53910058268492,
 39.19850747176416,
 64.1567377359908,
 56.906406424143405,
 50.376572998243276,
 35.29997708902531,
 23.06830068447856,
 35.933731379897296,
 22.105863013479816,
 22.171203389475878,

In [153]:
average_error_rf_check = Average(geo_dist_rf_check_m)

# Printing average of the list
print("Average RF error cross check [m] =", round(average_error_rf_check, 5))
print("Max RF error cross check [m] =", np.max(geo_dist_rf_check_m))
print("Min RF error cross check [m] =", np.min(geo_dist_rf_check_m))

Average RF error cross check [m] = 230.9503
Max RF error cross check [m] = 7092.641901140075
Min RF error cross check [m] = 0.6368453391485438
